In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
with open('data/aclImdb/models/TEXT.pkl','rb') as pklf:
    TEXT = pickle.load(pklf)

In [3]:
spacy_tok = spacy.load('en')

In [4]:
bs=32; bptt=20

In [5]:
PATH='data/'

TRN_PATH = 'kk_train/'
VAL_PATH = 'kk_valid/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

aclImdb/                    kk_train/  models/  wiki_en.txt  wiki_valid/
cleaned-extra-kk-jokes.txt  kk_valid/  tmp/     wiki_train/


In [6]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [7]:
print(
f'batches: {len(md.trn_dl)}\nunique tokens: {md.nt}\ntokens in training set: {len(md.trn_ds)}\nsentences: {len(md.trn_ds[0].text)}'
)

batches: 15
unique tokens: 37392
tokens in training set: 1
sentences: 10576


In [8]:
next(iter(md.trn_dl))

(Variable containing:
 
 Columns 0 to 10 
   3219   3219     61     50      4     61    658  16370     36  19065     61
      3      4   1049     45   2080   4282    171    686    137     43  11943
   3219     43   1049      0     43      4    337    167     10     61  11943
      4     16     43     45     61   4282     45    142      6  19065     43
     43     50     61      0   2080     43     38     45    881     27     61
     16     45   1049     43    605     61   3219     38     45    144  11943
     50      0     68     61      8   4282      3   3219     38   3251     36
     45     45      5      0   1232    263   3219      3   3219    182   1594
  25970      0   1611    431     68   4071      4   3219      3    144      3
     45     43     13    139     34     61     43      4   3219  16223     12
  25970     61   1235      2   4624     38     16     43      4      9     35
     43      0     45   4026      4   3219     50     16     43      6      8
     61  25799     38 

In [9]:
md.trn_ds[0].text[:10]

['knock', ',', 'knock', '.', 'who', "'s", 'there', '!', 'buggy', '!']

In [10]:
TEXT.numericalize([md.trn_ds[0].text[:10]])

Variable containing:
  3219
     3
  3219
     4
    43
    16
    50
    45
 25970
    45
[torch.cuda.LongTensor of size 10x1 (GPU 0)]

### Train

In [11]:
em_sz = 200  # size of each embedding vector
n_layers_act = 500     # number of hidden activations per layer
n_layers = 3      # number of layers

In [12]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [13]:
learner = md.get_model(opt_fn, em_sz, n_layers_act, n_layers,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.2

In [14]:
learner.load_encoder('adam3_20_enc')

In [15]:
learner.fit(3e-4, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      5.235847   5.216734  
    1      4.532408   4.382916                            
    2      4.1044     4.064408                            
    3      3.823516   3.878101                            
    4      3.56493    3.392389                            
    5      3.389188   3.524044                            
    6      3.218011   3.684946                            
    7      3.092188   3.006834                            
    8      2.942545   3.202374                            
    9      2.807659   3.344296                            
    10     2.681339   2.823181                            
    11     2.592538   3.27758                             
    12     2.505444   3.085213                            
    13     2.43974    3.180779                            
    14     2.39134    3.328322                            



[3.3283224]

In [16]:
learner.save_encoder('kkadam')

In [17]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                            
    0      2.310015   2.755177  
    1      2.168443   2.997514                            
    2      1.928563   3.209678                            
    3      1.727295   2.933261                            
    4      1.542455   3.260538                            
    5      1.42303    3.049026                            
    6      1.279217   2.913579                            
    7      1.18068    3.18208                             
    8      1.094555   3.182566                            
    9      1.025981   3.059655                            



[3.0596554]

In [18]:
learner.save_encoder('kkadam2')

In [19]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=13), HTML(value='')))

epoch      trn_loss   val_loss                             
    0      1.058636   3.116266  
    1      0.980902   2.953997                             
    2      0.910974   2.939238                             
    3      0.84777    3.141369                             
    4      0.827483   2.894783                             
    5      0.818164   3.151232                             
    6      0.78605    3.110808                             
    7      0.747835   3.299406                             
    8      0.70568    3.150657                             
    9      0.676542   3.109282                             
    10     0.669247   3.111122                             
    11     0.631094   3.330745                             
    12     0.60036    3.308752                             



[3.3087516]

### Test

In [36]:
m=learner.model
ss="""knock knock who's there"""
ss=''.join(w.lower() for w in ss)
s = [TEXT.tokenize(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

"knock knock who 's there"

In [37]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [38]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['?', '!', '.', '<eos>', 'there', '...', ',', 'who', 'you', 'here']

In [39]:
import random

In [51]:
tk=8
pct=.75
print(ss,"\n")
for i in range(100):
    if random.random()>pct:
        n=res[-1].topk(tk)[1]
        n = n[random.randint(0,tk-1)] if n.data[0]==0 else n[random.randint(0,tk-1)]
    else:
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

knock knock who's there 

<eos> knock , let 's there ? cheese cheese who who ? cheese some silly jokes ! <eos> knock , do you want one ? go out for a picnic ! <eos> knock , knock ... who 's there ! argo <unk> who ? argo down the shops for my money , just <unk> /><br knock , knock . who 's here ? bruce bruce who ? i bruce easily , do nt bother me ! <eos> knock , have ! ! <eos> knock , knock . and who 's there who ? annie - house ! <eos> let , ...
